We will learn to read VCF files within R using a publicly available dataset of genomic variant calls for the infamous individual, NA12878. The Genome-in-a-Bottle Consortium has compiled consensus variant calls on this individual's genome and released this data for researchers to use. One of the main purposes of this data is to provide a benchmark for those to develop computational tools and analysis of human genomes. See https://github.com/genome-in-a-bottle/giab_latest_release

Variant Call Format (VCF) is a very common format for representing genomic variation data. See Lecture 16: Slides 19.

## 0. Install and load the `VariantAnnotation` Bioconductor package 
Load the `VariantAnnotation` package

In [1]:
suppressPackageStartupMessages({
    library(VariantAnnotation)
    library(tidyverse)
})

## 1. Prepare parameters for reading VCF file.
There are a lot of variants in this file `GIAB_highconf_v.3.3.2.vcf.gz`, so we want to restrict to a smaller region for this example. 

### a. Setup parameters for scanning the VCF file.
First, we need to set up a `ScanVcfParam` object to read within `17:35500000-36000000`.

In [3]:
vcfFile <- "/Users/sgrizzi/Desktop/TFCB_data/GIAB_highconf_v.3.3.2.vcf.gz"
vcfHead <- scanVcfHeader(vcfFile)
myGRange4 <- GRanges(seqnames = "17", ranges = IRanges(start = 35500000, end = 36000000))
vcf.param <- ScanVcfParam(which = myGRange4) 

## 2. Read the VCF file.

In [4]:
vcf <- readVcf(vcfFile, genome = "hg19", param = vcf.param)

In [5]:
vcf

class: CollapsedVCF 
dim: 332 1 
rowRanges(vcf):
  GRanges with 5 metadata columns: paramRangeID, REF, ALT, QUAL, FILTER
info(vcf):
  DataFrame with 16 columns: DPSum, platforms, platformnames, platformbias, ...
info(header(vcf)):
                                   Number Type    Description                  
   DPSum                           1      Integer Total read depth summed ac...
   platforms                       1      Integer Number of different platfo...
   platformnames                   .      String  Names of platforms for whi...
   platformbias                    .      String  Names of platforms that ha...
   datasets                        1      Integer Number of different datase...
   datasetnames                    .      String  Names of datasets for whic...
   datasetsmissingcall             .      String  Names of datasets that are...
   callsets                        1      Integer Number of different callse...
   callsetnames                    .      String 

The `vcf` variable is of class `CollapsedVCF` and will contain header information and data. Let's see what information has been parsed by `readVcf`.

## 3. Extract the contents of the VCF entries.

### a. Return the variants in this region as a `GRanges` object.
The `rowRanges` function will return a `GRanges` object containing the coordinates, REF/ALT bases, quality, and filtering status of the variants.

In [9]:
#actual data of vcf files
rowRanges(vcf)

GRanges object with 332 ranges and 5 metadata columns:
              seqnames            ranges strand | paramRangeID            REF
                 <Rle>         <IRanges>  <Rle> |     <factor> <DNAStringSet>
    rs2411161       17          35501799      * |           NA              C
    rs8073074       17          35502949      * |           NA              A
    rs4523972       17          35507230      * |           NA              C
  rs111498996       17 35507465-35507466      * |           NA             CA
    rs8077266       17          35509302      * |           NA              A
          ...      ...               ...    ... .          ...            ...
    rs8080225       17          35996195      * |           NA              T
    rs8075378       17          35996582      * |           NA              G
    rs6607281       17          35997126      * |           NA              T
    rs4332783       17          35997674      * |           NA              A
   rs7198

### b. Inspect the header information
The `INFO` column in the original VCF text file contains a semi-colon delimited set of custom fields with flexible format that algorithms will output.  Here, it is parsed into usable format. First, let's look at what fields are available from the header.

In [12]:
info(vcf) 

DataFrame with 332 rows and 16 columns
                DPSum platforms       platformnames    platformbias  datasets
            <integer> <integer>     <CharacterList> <CharacterList> <integer>
rs2411161          NA         3   Illumina,CG,Solid                         3
rs8073074          NA         3   Illumina,CG,Solid                         4
rs4523972          NA         3   Illumina,CG,Solid                         3
rs111498996        NA         1            Illumina                         1
rs8077266          NA         3   Illumina,CG,Solid                         4
...               ...       ...                 ...             ...       ...
rs8080225          NA         4 Illumina,CG,10X,...                         4
rs8075378          NA         4 Illumina,CG,10X,...                         5
rs6607281          NA         4 Illumina,CG,10X,...                         4
rs4332783          NA         3     Illumina,CG,10X                         3
rs71984199         NA    

In [13]:
info(vcf) %>% # returns a DataFrame object
  as.data.frame() 

,DPSum,platforms,platformnames,platformbias,datasets,datasetnames,datasetsmissingcall,callsets,callsetnames,varType,filt,callable,difficultregion,arbitrated,callsetwiththisuniqgenopassing,callsetwithotheruniqgenopassing
,<int>,<int>,<list>,<list>,<int>,<list>,<list>,<int>,<list>,<chr>,<list>,<list>,<list>,<chr>,<list>,<list>
rs2411161,NA,3,Illumina....,,3,HiSeqPE3....,10XChrom....,4,HiSeqPE3....,NA,,CS_HiSeq....,,NA,,
rs8073074,NA,3,Illumina....,,4,HiSeqPE3....,10XChrom....,5,HiSeqPE3....,NA,CS_Solid....,CS_HiSeq....,,NA,,
rs4523972,NA,3,Illumina....,,3,HiSeqPE3....,10XChrom....,4,HiSeqPE3....,NA,CS_Solid....,CS_HiSeq....,,NA,,
rs111498996,NA,1,Illumina,,1,HiSeqPE300x,CGnormal....,2,HiSeqPE3....,NA,CS_CGnor....,CS_HiSeq....,AllRepea....,NA,,
rs8077266,NA,3,Illumina....,,4,HiSeqPE3....,10XChrom....,5,HiSeqPE3....,NA,CS_Solid....,CS_HiSeq....,,NA,,
rs75773263,NA,2,"CG, Illumina",,2,CGnormal....,10XChrom....,3,CGnormal....,NA,,CS_CGnor....,,NA,,
rs725038,NA,3,Illumina....,,4,HiSeqPE3....,10XChrom....,5,HiSeqPE3....,NA,CS_HiSeq....,CS_HiSeq....,,NA,,
rs10664252,NA,3,"CG, Illu....",,4,CGnormal....,10XChrom....,5,CGnormal....,NA,CS_Solid....,CS_CGnor....,,NA,,
rs11655013,NA,3,Illumina....,,3,HiSeqPE3....,10XChrom....,4,HiSeqPE3....,NA,,CS_HiSeq....,,NA,,


The `FORMAT` column in the original VCF text file contains the format and description of the genotype fields. Let's see what these are.

In [14]:
geno(header(vcf)) %>%
  as.data.frame() %>%
  rownames_to_column("colname")

colname,Number,Type,Description
<chr>,<chr>,<chr>,<chr>
GT,1,String,Consensus Genotype across all datasets with called genotype
DP,1,Integer,"Total read depth summed across all datasets, excluding MQ0 reads"
GQ,1,Integer,"Net Genotype quality across all datasets, calculated from GQ scores of callsets supporting the consensus GT, using only one callset from each dataset"
ADALL,R,Integer,Net allele depths across all datasets
AD,R,Integer,Net allele depths across all unfiltered datasets with called genotype
IGT,1,String,Original input genotype
IPS,1,String,Phase set for IGT
PS,1,String,Phase set for GT


### c. Inspect the genotype, read depth, and allele depth inforation.
To see the genotype `GT`, read depth `DP`, and allele depth `AD`, we access the the list.

In [7]:
geno(vcf)$GT[1:5]
geno(vcf)$DP[1:5]
geno(vcf)$AD[1:5]

[1] "1|1" "1|1" "1|1" "1|1" "1|1"

[1] 675 607 528 470 718

[[1]]
[1]  95 372

[[2]]
[1]  77 334

[[3]]
[1]  66 292

[[4]]
[1]   0 223

[[5]]
[1]  97 393

### d. Combine all `geno` fields into a single table.
You can also combine all fields into a `data.frame` object. But this code only works if the VCF contains a single sample.

In [15]:
genoData <- bind_cols(lapply(geno(vcf), as.data.frame))
colnames(genoData) <- rownames(geno(header(vcf)))
genoData[1:10,]

New names:
* `HG001` -> `HG001...1`
* `HG001` -> `HG001...2`
* `HG001` -> `HG001...3`
* `HG001` -> `HG001...4`
* `HG001` -> `HG001...5`
* `HG001` -> `HG001...6`
* `HG001` -> `HG001...7`
* `HG001` -> `HG001...8`


,GT,DP,GQ,ADALL,AD,IGT,IPS,PS
,<chr>,<int>,<int>,<named list>,<named list>,<chr>,<chr>,<chr>
rs2411161,1|1,675,405,"1, 301","95, 372",1/1,.,PATMAT
rs8073074,1|1,607,387,"0, 278","77, 334",1/1,.,PATMAT
rs4523972,1|1,528,290,"1, 230","66, 292",1/1,.,PATMAT
rs111498996,1|1,470,99,"0, 223","0, 223",1/1,.,PATMAT
rs8077266,1|1,718,452,"0, 328","97, 393",1/1,.,PATMAT
rs75773263,1|1,773,233,"0, 378","31, 31",1/1,.,PATMAT
rs725038,1|1,896,287,"0, 442","51, 51",1/1,.,PATMAT
rs10664252,1|1,577,366,"0, 265","68, 68",1/1,.,PATMAT
rs11655013,0|1,720,1197,"159, 155","206, 193",0/1,.,PATMAT


## Exercise 3: Reading variants from a VCF file.

### a. Create a range for `8:128747680-128753680`.

In [17]:
myGRange8 <- GRanges(seqnames = "8",
                    ranges = IRanges(start = 128747680, end = 128753680))

### b. Setup parameters to read VCF.

In [18]:
# ScanVcfParam
vcfFile <- "/Users/sgrizzi/Desktop/TFCB_data/GIAB_highconf_v.3.3.2.vcf.gz"
vcfHead <- scanVcfHeader(vcfFile)
vcf.param <- ScanVcfParam(which = myGRange8) 

### c. Read the VCF file at `8:128747680-128753680`

In [19]:
# readVcf
vcf_2 <- readVcf(vcfFile, genome = "hg19", param = vcf.param)

### d. What is the RS id, genotype (`GT`) and depth (`DP`) at the SNP in this locus?

In [23]:
# geno()
geno(vcf)$GT
geno(vcf)$DP

,HG001
rs2411161,1|1
rs8073074,1|1
rs4523972,1|1
rs111498996,1|1
rs8077266,1|1
rs75773263,1|1
rs725038,1|1
rs10664252,1|1
rs11655013,0|1
rs9906543,1|1


,HG001
rs2411161,675
rs8073074,607
rs4523972,528
rs111498996,470
rs8077266,718
rs75773263,773
rs725038,896
rs10664252,577
rs11655013,720
rs9906543,730
